In [3]:
import pandas as pd
import numpy as np

# Specify the path to your CSV file
file = r'C:\Users\user\Desktop\Analytics\August 2023\ranga t1\data-19.csv'

# Read the data from the file
df = pd.read_csv(file, low_memory=False)

# Function to handle mixed date formats
def parse_dates(date):
    try:
        return pd.to_datetime(date, format='%d-%m-%Y', errors='raise')
    except ValueError:
        return np.nan

# Apply the function to the 'openDate' column
df['openDate'] = df['openDate'].apply(parse_dates)

# Exclude rows with NaT in 'openDate' column
df = df[df['openDate'].notna()]

# Prepare the output DataFrame
output = df.groupby('cheqUserId')['openDate'].min().to_frame()

# Generate the required columns
months = pd.date_range(start="2021-08-01", end="2023-07-01", freq='MS')

for month in months:
    next_month = month + pd.offsets.MonthBegin(1)
    
    credit_line_col = f"Credit Line opened before {month.strftime('%B %Y')}"
    personal_loan_col = f"Personal Loan Opened in {month.strftime('%B %Y')}"
    
    output[credit_line_col] = (output['openDate'] < month).astype(int)
    
    mask = (df['accountTypeName'].str.contains('PERSONAL')) & (df['openDate'].dt.month == month.month) & (df['openDate'].dt.year == month.year)
    personal_loans = df.loc[mask].groupby('cheqUserId').size()
    output[personal_loan_col] = personal_loans
    output[personal_loan_col].fillna(0, inplace=True)

# Format the 'openDate' in the output DataFrame
output['openDate'] = output['openDate'].dt.strftime('%d-%m-%Y')

# Write the output DataFrame to a CSV file
output.to_csv(r'C:\Users\user\Desktop\Analytics\August 2023\ranga t1\output-data19.csv')


In [59]:
import pandas as pd

# Read the CSV files into pandas DataFrames
batch_5L_1 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data1.csv')
batch_5L_2 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data2.csv')
batch_5L_3 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data3.csv')
batch_5L_4 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data4.csv')
batch_5L_5 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data5.csv')
batch_5L_6 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data6.csv')
batch_5L_7 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data7.csv')
batch_5L_8 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data8.csv')
batch_5L_9 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data9.csv')
batch_5L_10 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data10.csv')
batch_5L_11 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data11.csv')
batch_5L_12 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data12.csv')
batch_5L_13 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data13.csv')
batch_5L_14 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data14.csv')
batch_5L_15 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data15.csv')
batch_5L_16 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data16.csv')
batch_5L_17 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data17.csv')
batch_5L_18 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data18.csv')
batch_5L_19 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data19.csv')
batch_5L_20 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data20.csv')
batch_5L_21 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data21.csv')
batch_5L_22 = pd.read_csv('C:/Users/user/Desktop/Analytics/ranga t1/output-data22.csv')

# Concatenate all DataFrames into one DataFrame
batch_5L = pd.concat([batch_5L_1, batch_5L_2, batch_5L_3, batch_5L_4, batch_5L_5, batch_5L_6, batch_5L_7, batch_5L_8, batch_5L_9, batch_5L_10, batch_5L_11, batch_5L_12, batch_5L_13, batch_5L_14, batch_5L_15, batch_5L_16, batch_5L_17, batch_5L_18, batch_5L_19, batch_5L_20, batch_5L_21, batch_5L_22], axis=0)

batch_5L.to_csv('C:/Users/user/Desktop/Analytics/ranga t1/final_output.csv', index=False)